In [1]:
# Note that this poc is based on the reply of Vitalik on EthResearch here:
# https://ethresear.ch/t/erc721-extension-for-zk-snarks/13237/13

# The code is not part of the zk-SNARK implementation an will be moved.

In [2]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account

$S = G*s$

In [3]:
# privkey: 0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30
# address: 0x3CB39EA2f14B16B69B451719A7BEd55e0aFEcE8F
s = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
S = secp256k1.privtopub(s.to_bytes(32, "big")) # public key
S

(22246744184454969143801186698733154500632648736073949898323976612504587645286,
 110772761940586493986212935445517909380300793379795289150161960681985511655321)

$P = G*p$

In [4]:
# privkey: 0x0000000000000000000000000000000000000000000000000000000000000001
# address: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
p = int(0x0000000000000000000000000000000000000000000000000000000000000002) # private key
P = secp256k1.privtopub(p.to_bytes(32, "big")) # public key
P

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

$P + G*hash(Q)$

In [5]:
Q = secp256k1.multiply(S, p)
assert Q == secp256k1.multiply(P, s)
Q_hex = sha3.keccak_256(Q[0].to_bytes(32, "big")+Q[1].to_bytes(32, "big")).hexdigest() # note, toStr conversion
Q_hased = bytearray.fromhex(Q_hex)

# Sender sends to ...
secp256k1.add(P, secp256k1.privtopub(Q_hased))

(34986710196311298046001056196012031484809340405385241652824827959002221322031,
 37835699564152814660978587905143437663484333033321007899868184134470225816011)

$p+hash(Q)$

In [6]:
p_stealth = p + int(Q_hex, 16)

# Recipient has private key to ...
P_stealth = secp256k1.privtopub(p_stealth.to_bytes(32, "big"))
P_stealth

(34986710196311298046001056196012031484809340405385241652824827959002221322031,
 37835699564152814660978587905143437663484333033321007899868184134470225816011)

In [7]:
Account.from_key((p_stealth).to_bytes(32, "big")).address

'0xa423e468c7987026D1dC797425a870C5e704E75d'